In [50]:
# Import libraries
import pandas as pd
import numpy as np


In [51]:
'''
Data properties: 8 classes (1,2,3,4,5,6,7,8), 8 features
X_train shape: (10800, 8)
y_train shape: (10800,)
X_test shape:  (1200, 8)
'''

train_data = pd.read_csv('./data/trn1.csv')
test_data = pd.read_csv('./data/tst1.csv', header=None)

# Prepare train and test data
X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values
X_test = test_data.values


C = len(np.unique(y_train)) # number of classes = 8
N = X_train.shape[0] # number of samples = 10800
d = X_train.shape[1] #number of features = 8

class_probs = np.array([np.mean(y_train == i) for i in range(1, C + 1)]) # (8,)
# print(class_probs.shape)

class_means = np.array([np.mean(X_train[y_train == i], axis=0) for i in range(1, C + 1)]) #(8, 8)
# print(class_means.shape)

class_cov = np.array([np.cov(X_train[y_train == i].T) for i in range(1, C + 1)]) #(8, 8, 8)
# print(class_cov.shape)


In [53]:

def discriminant_func(x, class_idx):
    x_diff = x - class_means[class_idx - 1] 
    cov_inv = np.linalg.inv(class_cov[class_idx - 1])   
    return -0.5 * np.log(np.linalg.det(class_cov[class_idx - 1])) - \
           0.5 * np.sum(x_diff @ cov_inv * x_diff, axis=1) + np.log(class_probs[class_idx - 1])

determinants = np.array([np.linalg.det(cov_matrix) for cov_matrix in class_cov])

discriminant_functions = np.array([discriminant_func(X_train, i) for i in range(1, C + 1)]).T
train_prediction = np.argmax(discriminant_functions, axis=1) + 1

accuracy = np.sum(train_prediction==y_train, axis=0)/y_train.shape[0]
print(f"Training Accuracy = {accuracy}")



Training Accuracy = 0.8969444444444444


In [55]:

discriminant_functions = np.array([discriminant_func(X_test, i) for i in range(1, C + 1)]).T
test_prediction = np.argmax(discriminant_functions, axis=1) + 1

# Add the predicted class to the testing data
result_data = np.column_stack((test_data, test_prediction))

# Save results to result1.csv
result_df = pd.DataFrame(result_data)
result_df.to_csv('./results/result1.csv', index=False, header=False)